### Capstone Project - The Battle of Neighborhoods

#### 1.Background

There are some research shows that New York City's demographics is a large and ethnically diverse metropolis. It is the largest city in the United States with a long history of international immigration. The population of New York City was nearly 8.5 million in 2014, accounting for over 40% of the population of New York State and a slightly lower percentage of the New York metropolitan area, home to approximately 23.6 million. Over the last decade the city has been growing faster than the region. The New York region continues to be by far the leading metropolitan gateway for legal immigrants admitted into the United States.

Due to New York's diversity, there are many kinds of restaurant such as Chinese, Indian, Italina,etc. In this project, I will list and visualize the major parts of New York City that has great Chinese resturants. Furthermore, the following questions will be answered: Where is the best location to open a Chinese restaurant in New York City? Which Chinese Resturant is the best in New York so far?


#### 2.Data

In this project, the following data will be used:

New York City data with list of Boroughs, Neighborhoods (latitude and longitude.) Data source : https://cocl.us/new_york_dataset
Description : This dataset contains the necessary information to explore various neighborhoods of new york city.

Chinese resturants in each neighborhood of new york city. Data source : Fousquare API
Description : Using this api to get all the venues in each neighborhood. After filtering these venues I will get Chinese resturants only.

GeoSpace data. Data source : https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
Description : In order to visualize the choropleth map, GeoSpace can help me to find the boundaries of New York Borough.

#### 3. Methodology


1. Import the new york city data from https://cocl.us/new_york_dataset to get the useful data.
2. Using FourSquare API to find all specific venues for each neighborhood.
3. Filtering out the venues to get Chinese Resturants.
4. Finding the most popular Restaurant by refering to rating, tips on FourSquare API.
5. Visualizing the ranking of neighborhoods by importing Python folium library.


#### 4. Data process, Analyses and Visualization

In [3]:
!pip install geocoder

     |████████████████████████████████| 102kB 18.2MB/s ta 0:00:01


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [87]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [111]:
wurl = requests.get("https://es.wikipedia.org/wiki/Anexo:Comunas_de_Chile_por_poblaci%C3%B3n").text
soup = BeautifulSoup(wurl,'lxml')

In [112]:
table_post = soup.find('table')
fields = table_post.find_all('td')

comuna = []
region = []
provincia = []
pob2017 = []
pos_nac = []
pos_reg = []
pos_pro = []
pob2002 = []
pob1992 = []



for i in range(0, len(fields), 9):
    comuna.append(fields[i].text.strip())
    region.append(fields[i+1].text.strip())
    provincia.append(fields[i+2].text.strip())
    pob2017.append(fields[i+3].text.strip())
    pos_nac.append(fields[i+4].text.strip())
    pos_reg.append(fields[i+5].text.strip())
    pos_pro.append(fields[i+6].text.strip())
    pob2002.append(fields[i+7].text.strip())
    pob1992.append(fields[i+8].text.strip())
   

df = pd.DataFrame(data=[region,comuna]).transpose()
df.columns = ['Borough','Neighborhood']
df.head()

,Borough,Neighborhood
0,Metropolitana de Santiago,Puente Alto
1,Metropolitana de Santiago,Maipú
2,Metropolitana de Santiago,Santiago
3,Metropolitana de Santiago,La Florida
4,Antofagasta,Antofagasta


In [113]:
df1=df[(df.Borough == 'Metropolitana de Santiago')]
df1.shape

(52, 2)

In [114]:
url = requests.get("https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_postales_de_Chile").text 
soup2 = BeautifulSoup(url,'lxml')

In [115]:
table = soup2.find('table')
ff = table.find_all('td')

comuna = []
codigo = []
                   
for i in range(0, len(ff), 2):
    comuna.append(ff[i].text.strip())
    codigo.append(ff[i+1].text.strip())
    
df2 = pd.DataFrame(data=[comuna,codigo]).transpose()
df2.columns = ['Neighborhood','PostalCode']
df2.head()

,Neighborhood,PostalCode
0,Algarrobo,2710000
1,Alhué,9650000
2,Alto Biobío,4590000
3,Alto del Carmen,1650000
4,Alto Hospicio,1130000


In [116]:
url="https://raw.githubusercontent.com/ssikam/My-Capstone-Project/master/chile%20geo%20public.csv"
df3 = pd.read_csv(url, encoding="ISO-8859-1", sep=";", names=["Pais", "Region", "Ciudad", "Neighborhood", "Lat", "Lon"])
df3.head()

,Pais,Region,Ciudad,Neighborhood,Lat,Lon
0,Chile,Antofagasta,Antofagasta,Antofagasta,-23.651,-70.395
1,Chile,Antofagasta,Antofagasta,Mejillones,-23.110,-70.456
2,Chile,Antofagasta,Antofagasta,Sierra Gorda,-22.898,-69.323
3,Chile,Antofagasta,Antofagasta,Taltal,-25.410,-70.489
4,Chile,Antofagasta,El Loa,Calama,-22.474,-68.924


In [97]:
dff = pd.merge(df1, df3[['Neighborhood','Lat', 'Lon']],
                       how='left', on=['Neighborhood'])
dff[39:52]

,Borough,Neighborhood,Lat,Lon
39,Metropolitana de Santiago,Talagante,-33.667,-70.931
40,Metropolitana de Santiago,Paine,-33.812,-70.723
41,Metropolitana de Santiago,Padre Hurtado,-33.576,-70.800
42,Metropolitana de Santiago,Isla de Maipo,-33.754,-70.886
43,Metropolitana de Santiago,El Monte,-33.684,-71.017
44,Metropolitana de Santiago,Curacaví,-33.399,-71.137
45,Metropolitana de Santiago,Pirque,-33.650,-70.564
46,Metropolitana de Santiago,Calera de Tango,-33.628,-70.785
47,Metropolitana de Santiago,Tiltil,-33.085,-70.925
48,Metropolitana de Santiago,San José de Maipo,-33.644,-70.353


In [98]:
dff.shape

(52, 4)

In [117]:
dff1 = dff.copy()
dff1.head(10)

,Borough,Neighborhood,Lat,Lon
0,Metropolitana de Santiago,Puente Alto,-33.616,-70.570
1,Metropolitana de Santiago,Maipú,-33.490,-70.788
2,Metropolitana de Santiago,Santiago,-33.425,-70.566
3,Metropolitana de Santiago,La Florida,-33.525,-70.538
4,Metropolitana de Santiago,San Bernardo,-33.582,-70.687
5,Metropolitana de Santiago,Las Condes,-33.400,-70.503
6,Metropolitana de Santiago,Peñalolén,-33.482,-70.538
7,Metropolitana de Santiago,Pudahuel,-33.411,-70.836
8,Metropolitana de Santiago,Quilicura,-33.361,-70.729
9,Metropolitana de Santiago,Ñuñoa,-33.454,-70.604


In [118]:
address = 'Santiago, Chile'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Santiago are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Santiago are -33.4377968, -70.6504451.


In [119]:
map_stgo = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, local in zip(dff1['Lat'], dff1['Lon'], dff1['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_stgo)  
    
map_stgo

In [126]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [127]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

In [128]:
stgo_venues_sushi = getNearbyVenues(names=dff1['Neighborhood'], latitudes=dff1['Lat'], longitudes=dff1['Lon'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
stgo_venues_sushi.head()

https://api.foursquare.com/v2/venues/search?&client_id=&client_secret=&v=&ll=-33.616,-70.57&radius=1000&limit=500&categoryId=4bf58dd8d48988d1d2941735
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'Value  is invalid for v', 'requestId': '5d82d0b76bdee60039cbd507'}, 'response': {}}


UnboundLocalError: local variable 'results' referenced before assignment

In [107]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [110]:
map_stgo_sushi = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(stgo_venues_sushi, 'red', map_stgo_sushi)
map_stgo_sushi

TypeError: 'Map' object is not subscriptable

In [57]:
stgo_venues_highschools = getNearbyVenues(names=dff1['Neighborhood'], latitudes=dff1['Lat'], longitudes=dff1['Lon'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
stgo_venues_highschools.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Puente Alto,-33.616,-70.570,Complejo Eduacional Consolidada,-33.611524,-70.571431,High School
1,Puente Alto,-33.616,-70.570,Colegio Polivalente Domingo Matte Mesias,-33.608298,-70.579750,High School
2,Puente Alto,-33.616,-70.570,Colegio Nidal,-33.611392,-70.573604,High School
3,Puente Alto,-33.616,-70.570,Colegio Tacora,-33.609515,-70.570331,High School
4,Maipú,-33.490,-70.788,Colegio Alicante El Rosal,-33.490642,-70.779636,High School


In [ ]:
map_stgo_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(stgo_venues_highschools, 'green', map_stgo_highschools)
map_stgo_highschools

In [59]:
stgo_venues_uni = getNearbyVenues(names=dff1['Neighborhood'], latitudes=dff1['Lat'], longitudes=dff1['Lon'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
stgo_venues_uni.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Santiago,-33.425,-70.566,Sala de Lenguaje,-33.425443,-70.569583,College Auditorium
1,Santiago,-33.425,-70.566,Sala de sonido-Incacea,-33.420655,-70.557696,University
2,Santiago,-33.425,-70.566,Casa Estudios,-33.418552,-70.566596,University
3,Santiago,-33.425,-70.566,Aula 3- Incacea,-33.417995,-70.556946,University
4,Las Condes,-33.400,-70.503,Escuela de Comunicaciones DuocUC,-33.400398,-70.505778,University


In [88]:
map_stgo_universities = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(stgo_venues_uni, 'gold', map_stgo_universities)
map_stgo_universities

In [61]:
stgo_venues_office = getNearbyVenues(names=dff1['Neighborhood'], latitudes=dff1['Lat'], longitudes=dff1['Lon'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
stgo_venues_office.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Puente Alto,-33.616,-70.57,Volcán Planta Concha y Toro,-33.620133,-70.573563,Office
1,Puente Alto,-33.616,-70.57,Bodega Constructora Vardero,-33.616335,-70.567696,Factory
2,Puente Alto,-33.616,-70.57,Integramédica,-33.612177,-70.576481,Medical Center
3,Puente Alto,-33.616,-70.57,CMPC Tissue S.A.,-33.611117,-70.565389,Office
4,Puente Alto,-33.616,-70.57,CMPC - Papeles Cordillera,-33.610262,-70.563704,Tech Startup


In [62]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0


In [63]:
df_data = dff1.copy()
df_data.rename(columns={'Neighborhood':'Localidad'}, inplace=True)
addColumn(df_data, 'Sushi', stgo_venues_sushi)
addColumn(df_data, 'High Schools', stgo_venues_highschools)
addColumn(df_data, 'Universities', stgo_venues_uni)
addColumn(df_data, 'Offices', stgo_venues_office)
df_data.head()


,Borough,Localidad,Lat,Lon,Sushi,High Schools,Universities,Offices
0,Metropolitana de Santiago,Puente Alto,-33.616,-70.570,6.0,4.0,0.0,47.0
1,Metropolitana de Santiago,Maipú,-33.490,-70.788,2.0,1.0,0.0,16.0
2,Metropolitana de Santiago,Santiago,-33.425,-70.566,6.0,9.0,4.0,50.0
3,Metropolitana de Santiago,La Florida,-33.525,-70.538,0.0,0.0,0.0,19.0
4,Metropolitana de Santiago,San Bernardo,-33.582,-70.687,3.0,6.0,0.0,50.0


In [64]:
# negative weight
weight_sushi = -1

# positive weight, ascending
weight_schools = 1
weight_uni = 2
weight_offices = 3

In [65]:
df_weighted = df_data[['Localidad']].copy()

In [66]:
df_weighted['Score'] = df_data['Sushi'] * weight_sushi + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted[39:52]

,Localidad,Score
41,Padre Hurtado,27.0
46,Calera de Tango,24.0
40,Paine,21.0
7,Pudahuel,12.0
50,San Pedro,6.0
45,Pirque,3.0
20,La Granja,3.0
38,Cerrillos,0.0
24,Lo Barnechea,0.0
37,San Ramón,0.0


In [89]:
map_stgo_result = folium.Map(location=[latitude, longitude], zoom_start=15)

stgo_win = dff1[dff1['Neighborhood'] == 'Providencia']

for lat, lng, local in zip(stgo_win['Lat'], stgo_win['Lon'], stgo_win['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_stgo_result) 

addToMap(stgo_venues_sushi[stgo_venues_sushi['Localidad'] == 'Providencia'], 'red', map_stgo_result)
addToMap(stgo_venues_highschools[stgo_venues_highschools['Localidad'] == 'Providencia'], 'green', map_stgo_result)
addToMap(stgo_venues_uni[stgo_venues_uni['Localidad'] == 'Providencia'], 'gold', map_stgo_result)
addToMap(stgo_venues_office[stgo_venues_office['Localidad'] == 'Providencia'], 'fuchsia', map_stgo_result)

map_stgo_result

TypeError: 'Map' object is not subscriptable

### 5.Results, Discussion and Conclusion

After finishing the project, I find that there are 137 Highschool, 138 Universities and 1654 Offices in Santiago. Also, the red dots in the second graph are competitors of Chinese restaurant. Therefore, the new restaurant should stay away from those places. On the other hand, the green dots and gold dots are highschool and universities, respectively. In order to make more profit, the new restaurant should be located around these dots.

Overall, taking all the effects into acccount, the best place to open a new restaurant should nearby the North-East of the Santiago.